# Informacje początkowe
Ten `python notebook` przedstawia działanie spejcalnego overlay'u dla platformy PYNQ który umożliwi nam przyszpieszenie operacji niektórych funkcji udostępnianych nam przez bibliotekę OpenCV, poprzez implementacje ich na układzie fpga.

Same operacje jakie będziemy sprawdzać to 2D filtry obrazu.

Obraz będzie pobierany na bierząco z wejścia hdmi płytki i wypuszczany przez wyjście hdmi płytki.

Ten notebook posiada następujące kroki:
1. Inicjalizacja płytki oraz fpga
2. Pokazanie działania filtrowania obrazu przy pomocy procesora
3. Pokazanie działanie filtrowania obrazu przy pomocy fpga

# Inicjalizacja płytki oraz fpga

## Ładowanie overlay'u
Ładujemy specjalny overlay który pozwoli nam używać funkcji openCV na układzie fpga, co przyspieszy ich działanie.

In [ ]:
# Load filter2D + dilate overlay
from pynq import Overlay
bareHDMI = Overlay("/usr/local/lib/python3.6/dist-packages/"
               "pynq_cv/overlays/xv2Filter2DDilate.bit")
import pynq_cv.overlays.xv2Filter2DDilate as xv2

# Load xlnk memory mangager
from pynq import Xlnk
Xlnk.set_allocator_library("/usr/local/lib/python3.6/dist-packages/"
                           "pynq_cv/overlays/xv2Filter2DDilate.so")
mem_manager = Xlnk()

## Konfiguracja hdmi

In [ ]:
hdmi_in = bareHDMI.video.hdmi_in
hdmi_out = bareHDMI.video.hdmi_out

from pynq.lib.video import *
hdmi_in.configure(PIXEL_GRAY)
hdmi_out.configure(hdmi_in.mode)

hdmi_in.cacheable_frames = False
hdmi_out.cacheable_frames = False

hdmi_in.start()
hdmi_out.start()

## Konfiguracja parametrów wejścia i wyjścia dla hdmi 

In [ ]:
mymode = hdmi_in.mode
print("My mode: " + str(mymode))

height = hdmi_in.mode.height
width = hdmi_in.mode.width
bpp = hdmi_in.mode.bits_per_pixel

# Filtrowanie obrazu przy pomocy procesora

Funkcja dzięki której będziemy móc zmieniać dynamicznie filtry obrazu.
Do demonstracji działania wykorzystaliśmy 6 filtrów.

In [ ]:
import numpy as np
  
def setKernelAndFilter3x3(kernelName):
    global kernel_g

    kernel_g = {
        'Laplacian high-pass': np.array([[0.0,1.0,0.0],[1.0,-4.0,1.0],
                                         [0.0,1.0,0.0]],np.float32),
        'Gaussian high-pass': np.array([[-0.0625,-0.125,-0.0625],
                                        [-0.125,0.75,-0.125],
                                        [-0.0625,-0.125,-0.0625]],np.float32),
        'Average blur':  np.ones((3,3),np.float32)/9.0,
        'Gaussian blur': np.array([[0.0625,0.125,0.0625],
                                   [0.125,0.25,0.125],
                                   [0.0625,0.125,0.0625]],np.float32),
        'Sobel ver': np.array([[1.0,0.0,-1.0],[2.0,0.0,-2.0],
                               [1.0,0.0,-1.0]],np.float32),
        'Sobel hor': np.array([[1.0,2.0,1.0],[0.0,0.0,0.0],
                               [-1.0,-2.0,-1.0]],np.float32)
    }.get(kernelName, np.ones((3,3),np.float32)/9.0)

Funkcja dzięki będziemy mogli dynamicznie zmieniać czy filtrowanie ma być przeprowadzane przez procesor czy fpga 

In [ ]:
def setProcessing(whichProcessing):
    global processing_g
    
    processing_g = {
        'Fpga processing': 'fpga',
        'Procesor processing': 'procesor'
    }.get(whichProcessing)

### Główna funkcja
Główna funkcja która jest odpowiedzialna za pobranie obrazu, nałożeniu filtru i wypuszczeniu przetworzonego obrazu na wyjście.

In [ ]:
import cv2

def imageProccesing(fps):
    outframe = hdmi_out.newframe()   # Pobieramy klatkę strukturę klatki by móc ją później wykorzystać 
    inframe = hdmi_in.readframe()    # Pobranie klatki z wejścia
    
    # Nałożenie filtru
    if processing_g == 'fpga':
        xv2.filter2D(inframe, -1, kernel_g, dst=outframe, borderType=cv2.BORDER_CONSTANT)   # Filtr przy użyciu fpga
    elif processing_g == 'procesor':
        cv2.filter2D(inframe, -1, kernel_g, dst=outframe,  borderType=cv2.BORDER_CONSTANT)   # Filtr przy użyciu procesora
   
    font                   = cv2.FONT_HERSHEY_SIMPLEX
    bottomLeftCornerOfText = (1020,1060)
    fontScale              = 1
    fontColor              = (255,255,255)
    lineType               = 2
    
    cv2.putText(outframe,str("%.2f" % fps), 
    bottomLeftCornerOfText, 
    font, 
    fontScale,
    fontColor,
    lineType)
    
    hdmi_out.writeframe(outframe)    # Wypuszczenie przetworzonego obrazu
    inframe.freebuffer()             # Wyczyszczenie bufora

### Klasa która umożliwia nam dynamiczne zmienianie filtra
Sama klasa jest dość prosta gdyż jest to wątek który w pętli wykonuje przetwarzanie obrazu dopóki ktoś jej nie zastopuje metodą `stop()`

In [ ]:
import threading
import time
import statistics 
from IPython.display import display, clear_output


class MainLoop(threading.Thread):
    def __init__(self):
        super(MainLoop, self).__init__()
        self.flag = True
        
    def run(self):
        num_frames = 60
        frame = 0
        frame_table = [0] * 60
        table_sum = 0
        fps = 0
        
        while self.flag:
            start = time.time()
            imageProccesing(fps)
            end = time.time()
            elapsed = end - start
            table_sum += elapsed
            table_sum -= frame_table[frame]
            frame_table[frame] = elapsed
            frame = (frame + 1) % 60
            
            fps = 1 / (table_sum / 60)
            
            
            
    def stop(self):
        self.flag = False

### Rozpoczęcie działania programu 
Po rozpoczęciu możemy zmieniać dynamicznie jakim filtrem chcemy zmienić obraz,
do tego służy nam specjalny interaktywny widget ipython 

#### Czy ma przetwarzać procesor czy fpga

In [ ]:
from ipywidgets import interact, interactive, fixed, interact_manual
from ipywidgets import IntSlider, FloatSlider
import ipywidgets as widgets

interact(setProcessing, whichProcessing = ['Fpga processing', 'Procesor processing']);

#### Jaki filtr

In [ ]:
interact(setKernelAndFilter3x3, kernelName
         = ['Sobel ver','Sobel hor','Laplacian high-pass','Gaussian high-pass','Average blur',
            'Gaussian blur',]);

### Rozpoczęcie przetwarzania

In [ ]:
t = MainLoop()
t.start()

### Zatrzymanie działania programu

In [ ]:
t.stop()

### Zamknij hdmi

In [ ]:
hdmi_out.close()
hdmi_in.close()